In [1]:
import os
os.chdir("../../")


In [2]:
import numpy as np
import pandas as pd
from src.models import spread_model

In [3]:
list_of_simulation = [spread_model.simulate_pandemic(p_r = 0.5, lambda_leak=.01, pop_size=5000, 
                                initial_infection = 1/5000) for x in range(5)]

In [4]:
from joblib import Parallel, delayed
sims_caveman = Parallel(n_jobs=-1)(delayed(spread_model.simulate_pandemic)(p_r = 0.5,
                                                                        lambda_leak=.01,
                                                                        pop_size=5000,
                                                initial_infection = 1/5000) for i in range(100))

In [6]:
for col in ['infected']:
    tmp=[sims_caveman[i][col] for i in range(len(sims_caveman))]
    infected_sims = pd.DataFrame(tmp).T.fillna(0)
    infected_sims['mean'] = infected_sims.apply(np.mean, axis=1)
    infected_sims['sd'] = infected_sims.drop(columns='mean').apply(np.std, axis=1)
    infected_sims_caveman = infected_sims.drop(columns=col)

In [7]:
from joblib import Parallel, delayed
sims_scalefree = Parallel(n_jobs=-1)(delayed(spread_model.simulate_pandemic)(p_r = 0.5,
                                                                        lambda_leak=.01,
                                                                        pop_size=5000,
                                                                        graph_model='scale_free',
                                                initial_infection = 1/5000) for i in range(100))

In [12]:
for col in ['infected']:
    tmp=[sims_scalefree[i][col] for i in range(len(sims_scalefree))]
    infected_sims = pd.DataFrame(tmp).T.fillna(0)
    infected_sims['mean'] = infected_sims.apply(np.mean, axis=1)
    infected_sims['sd'] = infected_sims.drop(columns='mean').apply(np.std, axis=1)
    infected_scalefree = infected_sims.drop(columns=col)

In [13]:
import plotly.graph_objects as go

fig = go.Figure()

x = infected_sims_caveman.index.to_list()[:100]
y = infected_sims_caveman['mean'][:100]
sd = infected_sims_caveman['sd'][:100]


fig.add_trace(go.Scatter(x=x, y=y,
    fill=None,
    mode='lines',
    line_color='indigo',
     name="Average Curve"
    ))

fig.add_trace(go.Scatter(x=x, y=y+sd,
    fill=None,
    mode='lines',
    line_color='magenta',
    showlegend=False
   ))

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='indigo',      name="Standard Deviation"))

fig.add_trace(go.Scatter(x=x, y=y-sd,
    fill=None,
    mode='lines',
    line_color='magenta',    showlegend=False
    ))


fig.add_trace(go.Scatter(
    x=x,
    y=y,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='indigo',     showlegend=False
))

fig.update_layout(title = 'Average infection Curve - Relaxed Caveman')

fig.show()

In [14]:
import plotly.graph_objects as go

fig = go.Figure()

x = infected_scalefree.index.to_list()[:120]
y = infected_scalefree['mean'][:120]
sd = infected_scalefree['sd'][:120]


fig.add_trace(go.Scatter(x=x, y=y,
    fill=None,
    mode='lines',
    line_color='indigo',
     name="Average Curve"
    ))

fig.add_trace(go.Scatter(x=x, y=y+sd,
    fill=None,
    mode='lines',
    line_color='magenta',
    showlegend=False
   ))

fig.add_trace(go.Scatter(
    x=x,
    y=y,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='indigo',      name="Standard Deviation"))

fig.add_trace(go.Scatter(x=x, y=y-sd,
    fill=None,
    mode='lines',
    line_color='magenta',    showlegend=False
    ))


fig.add_trace(go.Scatter(
    x=x,
    y=y,
    fill='tonexty', # fill area between trace0 and trace1
    mode='lines', line_color='indigo',     showlegend=False
))


fig.update_layout(title = 'Average infection Curve - Scale Free')

fig.show()